In [1]:
import numpy as np
from abc import ABC, abstractmethod
from collections import deque
from numpy_ringbuffer import RingBuffer
from functools import partial

In [2]:
class sndaqbuffer(ABC):
    def __init__(self, size, ndom=5160, dtype=np.uint16):
        self._size = size
        self._ndom = ndom
        self._dtype = dtype 
        super().__init__()
        
    @abstractmethod
    def append(self, entry):
        ...
        
    @abstractmethod
    def clear(self):
        ...
    
    def __getitem__(self, key):
        return self.data[key]
    
    @property
    @abstractmethod
    def data(self):
        ...


In [3]:
class rollingbuffer(sndaqbuffer):
    def __init__(self, size, ndom=5160, dtype=np.uint16):
        super().__init__(size, ndom, dtype)
        self.clear()
    
    def append(self, entry):
        self._data = np.roll(self._data, -11, axis=0)
        self._data[-1,:] = entry
    
    def clear(self):
        self._data = np.zeros(shape=(self._size, self._ndom),
                              dtype=self._dtype)
    @property
    def data(self):
        return self._data
    

In [4]:
class windowbuffer(sndaqbuffer):
    def __init__(self, size, ndom=5160, dtype=np.uint16, mult=2):
        super().__init__(size, ndom, dtype)
        self._mult = mult
        self._buflen = self._size*self._mult
        self.clear()
    
    def append(self, entry):
        if self._idx >= self._buflen:
            self._reset()
        self._data[self._idx,:] = entry
        self._idx += 1
        return self
            
    def clear(self):
        self._data = np.zeros(shape=(self._buflen, self._ndom),
                              dtype=self._dtype)
        self._idx = self._size
    
    def _reset(self):
        self._idx = self._size
        self._data[:self._size,:] = self._data[-self._size:,:]
        
    def __getitem__(self, key):
        return self.data[key]
        
    @property
    def data(self):
        return self._data[self._idx-self._size:self._idx,:]

In [5]:
class ringbuf(sndaqbuffer):
    def __init__(self, size, ndom=5160, dtype=np.uint16):
        super().__init__(size, ndom, dtype)
        self.clear()
        
    def append(self, entry):
        self._data.append(entry)
        return self
        
    def clear(self):
        self._data = RingBuffer(capacity=size, dtype=(self._dtype, self._ndom))
        
    @property
    def data(self):
        return self._data



In [6]:
size = int(90/.002)
buffers = []
# buffers.append(rollingbuffer(size))  # WAY TOO SLOW
buffers.append(windowbuffer(size))
buffers.append(ringbuf(size))
rcol = partial(np.random.randint, low=100, dtype=np.uint16, size=5160)

for buf in buffers:
    print(type(buf).__name__ + " Append data")
    %timeit -n1 -r3 for i in range(int(size*1.5)): buf.append(rcol())

for buf in buffers:
    print(type(buf).__name__ + " Append + get data")
    %timeit -n1 -r3 for i in range(int(size*1.5)): buf.append(rcol()).data

for buf in buffers[:-1]: # Too slow for ringbuf (doesn't finish)
    print(type(buf).__name__ + " Append + get slice of data")
    %timeit -n1 -r3 for i in range(int(size*1.5)): buf.append(rcol())[0:size//4]


windowbuffer Append data
2.53 s ± 41.9 ms per loop (mean ± std. dev. of 3 runs, 1 loop each)
ringbuf Append data
2.57 s ± 9.65 ms per loop (mean ± std. dev. of 3 runs, 1 loop each)
windowbuffer Append + get data
2.57 s ± 17 ms per loop (mean ± std. dev. of 3 runs, 1 loop each)
ringbuf Append + get data
2.6 s ± 25.7 ms per loop (mean ± std. dev. of 3 runs, 1 loop each)
windowbuffer Append + get slice of data
2.64 s ± 36 ms per loop (mean ± std. dev. of 3 runs, 1 loop each)
